Vehicle Sales Data

**Données** :

Source : Kaggle “Vehicle Sales Data” : https://www.kaggle.com/datasets/syedanwarafridi/vehicle-sales-data

"year": row[0]
"make": row[1]
"model": row[2]
"trim": row[3]
"body": row[4]
"transmission": row[5]
"vin": row[6]
"state": row[7]
"condition": row[8]
"odometer": row[9]
"color": row[10]
"interior": row[11]
"seller": row[12]
"mmr": row[13]
"sellingprice": row[14]
"saledate": row[15]

## Énoncé

depuis le dataset afficher :

1. Véhicule le plus cher
2. Véhicule le moins cher
3. Kilométrage extrêmes (minimum et maximum)
4. Marque la plus vendue
5. Modèle le plus vendu par État “CA”
6. Années extrêmes (minimum et maximum)
7. **Bonus** la voiture avec le plus gros moteur


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").getOrCreate()
sc = spark.sparkContext

In [3]:
rdd = sc.textFile("../Data/car_prices.csv")
rdd.count()

558838

In [5]:
header = rdd.first()
data = rdd.filter(lambda line : line != header)
data.count()

558837

In [6]:
# 1.
def most_expensive(line):
    line = line.split(",")
    make = line[1]
    model = line[2]
    try:
        price = float(line[14])
        return (price, make, model)
    except:
        return (0.0, make, model)
    
data.map(most_expensive).filter(lambda x: x[0] > 0).sortBy(lambda x: x[0], ascending=False).first()

(230000.0, 'Ford', 'Escape')

In [8]:
# 2.
data.map(most_expensive).filter(lambda x: x[0] > 0).sortBy(lambda x: x[0]).first()

(1.0, 'Ford', 'F-350 Super Duty')

In [9]:
def get_mileage(line):
    line = line.split(",")
    make = line[1]
    model = line[2]
    try:
        mileage = float(line[9])
        return (mileage, make, model)
    except:
        return (0.0, make, model)
    
data.map(get_mileage).filter(lambda x: x[0] > 0).sortBy(lambda x: x[0], ascending=False).first()

(999999.0, 'Hyundai', 'Elantra Coupe')

In [10]:
data.map(get_mileage).filter(lambda x: x[0] > 0).sortBy(lambda x: x[0]).first()

(1.0, 'Nissan', 'Maxima')

In [ ]:
# Marque la plus vendue
def get_make(line):
    line = line.split(",")
    return line[1]

data.map(get_make).filter(lambda x : x != "") \
.map(lambda x: (x, 1)) \
.reduceByKey(lambda a,b : a + b) \
.sortBy(lambda x : x[1], ascending=False).first()

('Ford', 93554)

In [13]:
data.map(get_make).filter(lambda x : x != "") \
.map(lambda x: (x, 1)) \
.reduceByKey(lambda a,b : a + b) \
.sortBy(lambda x : x[1]).first()

('Lotus', 1)

In [14]:
def get_model_ca(line):
    line = line.split(",")
    state = line[7]
    model = line[2]
    if state.lower() == "ca":
        return model
    else:
        return ""

data.map(get_model_ca).filter(lambda x : x != "").map(lambda x : (x, 1)) \
.reduceByKey(lambda a,b : a + b) \
.sortBy(lambda x : x[1], ascending=False).first()

('Altima', 2787)

In [19]:
def get_year(line):
    line = line.split(",")
    try:
        year = int(line[0])
        return year
    except:
        return 0
    
data.map(get_year).filter(lambda x : x > 0).max()

2015

In [20]:
data.map(get_year).filter(lambda x : x > 0).min()

1982